Solving Package delivery using single-agent PPO with a naive feature representation learning: concatenante all the feature in to a single state vector, and multiple robot actions as a multi discrete distribution.

In [13]:
%%capture
!git clone https://github.com/cuongtv312/marl-delivery.git
%cd marl-delivery
!pip install -r requirements.txt

In [14]:
%%capture
!pip install stable-baselines3

In [20]:
from env import Environment
import gymnasium as gym
from gymnasium import spaces
import numpy as np

In [ ]:
# TODO: Modify this one to add more information to the Agents
def convert_state(state):
    # Get basic information
    robots_info = np.array(state["robots"]).astype(np.float32)  # Shape: (n_robots, 3)
    packages_info = np.array(state["packages"]).astype(np.float32) if state["packages"] else np.array([])
    
    n_robots = len(robots_info)
    features = []
    
    # For each robot, compute relevant features
    for i in range(n_robots):
        robot_pos = robots_info[i, :2]  # x, y position
        carrying = robots_info[i, 2]    # package ID being carried
        
        robot_features = [robot_pos[0], robot_pos[1], carrying]
        
        if carrying > 0:  # Robot is carrying a package
            # Find target location of the carried package
            carried_pkg_idx = int(carrying) - 1
            for pkg in packages_info:
                if pkg[0] == carrying:  # package ID matches
                    target_pos = pkg[3:5]  # target position
                    # Distance to target
                    dist_to_target = np.sqrt((robot_pos[0] - target_pos[0])**2 + 
                                            (robot_pos[1] - target_pos[1])**2)
                    robot_features.extend([target_pos[0], target_pos[1], dist_to_target, 0, 0, 0])
                    break
            else:  # Package already delivered or not found
                robot_features.extend([0, 0, 0, 0, 0, 0])
        else:  # Robot is not carrying a package
            # Find closest waiting package
            min_dist = float('inf')
            closest_pkg_pos = [0, 0]
            closest_pkg_target = [0, 0]
            
            for pkg in packages_info:
                pkg_id, start_x, start_y = pkg[0], pkg[1], pkg[2]
                target_x, target_y = pkg[3], pkg[4]
                
                # Check if package is waiting for pickup
                dist = np.sqrt((robot_pos[0] - start_x)**2 + (robot_pos[1] - start_y)**2)
                if dist < min_dist:
                    min_dist = dist
                    closest_pkg_pos = [start_x, start_y]
                    closest_pkg_target = [target_x, target_y]
            
            if min_dist != float('inf'):
                robot_features.extend([closest_pkg_pos[0], closest_pkg_pos[1], min_dist,
                                      closest_pkg_target[0], closest_pkg_target[1], 0])
            else:
                robot_features.extend([0, 0, 0, 0, 0, 0])
        
        features.extend(robot_features)
    
    # Add global information: total packages waiting/in transit
    if len(packages_info) > 0:
        features.append(len(packages_info))
    else:
        features.append(0)
    
    return np.array(features, dtype=np.float32)

In [ ]:
# TODO: Modify this one to make the agent learn faster

def reward_shaping(r, env, state, action):
    shaped_reward = r  # Start with the original reward
    
    # Get current state information
    robots_info = np.array(state["robots"]).astype(np.float32)
    packages_info = np.array(state["packages"]).astype(np.float32) if state["packages"] else np.array([])
    
    # Track robot movements to penalize standing still without purpose
    for i, robot in enumerate(env.robots):
        robot_pos = robot.position
        carrying = robot.carrying
        move_action, pkg_action = action[i]
        
        # 1. Penalize robots that stand still without delivering enough goods
        if move_action == 'S' and carrying == 0:
            shaped_reward -= 0.05  # Small penalty for standing still without package
        
        # 2. Robots that move will be penalized for moving costs
        # This is already handled in the environment, so we don't need to add it here
        
        # 3. Robot without package: reward getting closer to nearest package
        if carrying == 0:
            # Find closest package
            min_dist = float('inf')
            prev_min_dist = float('inf')
            for pkg in packages_info:
                pkg_id, start_x, start_y = pkg[0], pkg[1]-1, pkg[2]-1  # Adjust for 1-indexing
                
                curr_dist = np.sqrt((robot_pos[0] - start_x)**2 + (robot_pos[1] - start_y)**2)
                
                # Calculate previous position
                prev_pos = env.compute_new_position(robot_pos, 'S')  # Use 'S' to get current position
                if move_action == 'L':
                    prev_pos = env.compute_new_position(robot_pos, 'R')
                elif move_action == 'R':
                    prev_pos = env.compute_new_position(robot_pos, 'L')
                elif move_action == 'U':
                    prev_pos = env.compute_new_position(robot_pos, 'D')
                elif move_action == 'D':
                    prev_pos = env.compute_new_position(robot_pos, 'U')
                
                prev_dist = np.sqrt((prev_pos[0] - start_x)**2 + (prev_pos[1] - start_y)**2)
                
                if curr_dist < min_dist:
                    min_dist = curr_dist
                    prev_min_dist = prev_dist
            
            # Reward/penalize based on distance change
            if min_dist < prev_min_dist:
                shaped_reward += 0.1  # Getting closer to package
            elif min_dist > prev_min_dist:
                shaped_reward -= 0.1  # Moving away from package
            
            # Bonus for picking up a package
            if pkg_action == '1':
                shaped_reward += 0.5
        
        # 4. Robot with package: reward getting closer to target
        else:  # carrying > 0
            package_id = carrying
            package_target = None
            
            # Find the target of the package being carried
            for pkg in packages_info:
                if pkg[0] == package_id:
                    target_x, target_y = pkg[3]-1, pkg[4]-1  # Adjust for 1-indexing
                    package_target = (target_x, target_y)
                    break
            
            if package_target:
                # Calculate current and previous distance to target
                curr_dist = np.sqrt((robot_pos[0] - package_target[0])**2 + 
                                    (robot_pos[1] - package_target[1])**2)
                
                # Calculate previous position
                prev_pos = env.compute_new_position(robot_pos, 'S')
                if move_action == 'L':
                    prev_pos = env.compute_new_position(robot_pos, 'R')
                elif move_action == 'R':
                    prev_pos = env.compute_new_position(robot_pos, 'L')
                elif move_action == 'U':
                    prev_pos = env.compute_new_position(robot_pos, 'D')
                elif move_action == 'D':
                    prev_pos = env.compute_new_position(robot_pos, 'U')
                
                prev_dist = np.sqrt((prev_pos[0] - package_target[0])**2 + 
                                   (prev_pos[1] - package_target[1])**2)
                
                # Reward/penalize based on distance change
                if curr_dist < prev_dist:
                    shaped_reward += 0.2  # Getting closer to target
                elif curr_dist > prev_dist:
                    shaped_reward -= 0.2  # Moving away from target
    
    return r + shaped_reward

In [25]:
# Avoid to modify the Env class,
# If it is neccessary, you should describe those changes clearly in report and code
class Env(gym.Env):
    def __init__(self, *args, **kwargs):
        super(Env, self).__init__()
        self.env = Environment(*args, **kwargs)

        self.action_space = spaces.multi_discrete.MultiDiscrete([5, 3]*self.env.n_robots)


        self.prev_state = self.env.reset()
        first_state=convert_state(self.prev_state)
        # Define observation space as a dictionary

        self.observation_space = spaces.Box(low=0, high=100, shape=first_state.shape, dtype=np.float32)


        from sklearn.preprocessing import LabelEncoder
        self.le1, self.le2= LabelEncoder(), LabelEncoder()
        self.le1.fit(['S', 'L', 'R', 'U', 'D'])
        self.le2.fit(['0','1', '2'])

    def reset(self, *args, **kwargs):
        self.prev_state = self.env.reset()
        return convert_state(self.prev_state), {}

    def render(self, *args, **kwargs):
        return self.env.render()

    def step(self, action):
        ret = []
        ret.append(self.le1.inverse_transform(action.reshape(-1, 2).T[0]))
        ret.append(self.le2.inverse_transform(action.reshape(-1, 2).T[1]))
        action = list(zip(*ret))

        # You should not modify the infos object
        s, r, done, infos = self.env.step(action)
        new_r = reward_shaping(r, self.env, self.prev_state, action)
        self.prev_state = s
        return convert_state(s), new_r, \
            done, False, infos

In [26]:
import gymnasium as gym

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import EvalCallback


# Parallel environments
vec_env = make_vec_env(lambda: Env('map2.txt', 100, 5, 20, -0.01, 10., 1., 10), n_envs=10)
eval_env = Monitor(Env('map2.txt', 100, 5, 20, -0.01, 10., 1., 10), "ppo_delivery")

eval_callback = EvalCallback(eval_env, best_model_save_path="./best_model/",
                             log_path="./logs/", eval_freq=5000,
                             deterministic=True, render=False)

model = PPO("MlpPolicy", vec_env, verbose=1)
model.learn(total_timesteps=10000, callback=eval_callback)
model.save("ppo_delivery")


Using cpu device
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 100      |
|    ep_rew_mean     | -2.68    |
| time/              |          |
|    fps             | 1017     |
|    iterations      | 1        |
|    time_elapsed    | 20       |
|    total_timesteps | 20480    |
---------------------------------


In [27]:
obs,_ = eval_env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, dones, _, info = eval_env.step(action)
    #print('='*10)
    #eval_env.unwrapped.env.render()
    if dones:
        break

print(info)

{'total_reward': -3.339999999999998, 'total_time_steps': 100, 'episode': {'r': -3.34, 'l': 100, 't': 101.408837}}


In [7]:
!pip freeze | grep stable_baselines3

stable_baselines3==2.6.0
